# STATIC STUFF 
 
THIS PART IS USED FOR THE SETUP OF THIS NOTEBOOK

Establish connection

In [1]:
from modules.athena_queries import AthenaQueries
import pandas as pd
q = AthenaQueries()

Test connection|

In [2]:
df = q.athena_query('SELECT * FROM "default"."details" limit 10;')
df

,id,lng,lat,name,county,parish,abs_price,abs_size,title,room_nr,url,partition_0,partition_1,partition_2,partition_3
0,3117574,26.3455648,59.3543063,Rakvere,7,1050,125135,86,,4,https://www.kv.ee/3117574,sale,Rakvere,2020-04-02,4
1,3000657,26.368445467565,59.346103567358,Rakvere,7,1050,57500,77,,4,https://www.kv.ee/3000657,sale,Rakvere,2020-04-02,4
2,3109178,26.3678178,59.3570287,Rakvere,7,1050,85000,120,UUS HIND! MÜÜGIGA KIIRE! SINU UUS KODU OOTAB S...,4,https://www.kv.ee/3109178,sale,Rakvere,2020-04-02,4
3,3220362,26.3432167,59.353984,Rakvere,7,1050,95500,81,Uus korter!,4,https://www.kv.ee/3220362,sale,Rakvere,2020-04-02,4
4,3217155,24.6601409,59.4009599,Tallinn,1,1061,400,49,,3,https://www.kv.ee/3217155,rent,Tallinn,2020-02-28,3
5,3215948,24.7535747,59.4369608,Tallinn,1,1061,1600,75,Kesklinn! Disainkorter! Parkimiskoht hinnas!,3,https://www.kv.ee/3215948,rent,Tallinn,2020-02-28,3
6,3197231,24.7468124,59.4407318,Tallinn,1,1061,3500,200,Eksklusiivne 3-toaline disainkorter Vanalinnas,3,https://www.kv.ee/3197231,rent,Tallinn,2020-02-28,3
7,3204264,24.7791448,59.4391391,Tallinn,1,1061,750,47,ESINDUSLIK KODU KADRIOURU JA KESKLINNA PIIRIL!,3,https://www.kv.ee/3204264,rent,Tallinn,2020-02-28,3
8,3192385,24.8245712,59.4644374,Tallinn,1,1061,950,106,"3-toaline korter Pirital, vaatega vanalinnale!",3,https://www.kv.ee/3192385,rent,Tallinn,2020-02-28,3
9,2830513,24.6444173,59.3974298,Tallinn,1,1061,550,67,,3,https://www.kv.ee/2830513,rent,Tallinn,2020-02-28,3


In [3]:
%load_ext autoreload
%autoreload

# PRICE CHANGES FOR CERTAIN APARTMENTS

In [4]:
price_changes = q.get_price_differences("sale")

In [17]:
df = price_changes

In [18]:
df.dtypes

url             object
room_nr          int64
title           object
partition_1     object
abs_size         int64
abs_price        int64
prev_price     float64
dt              object
dtype: object

In [ ]:
def get_items_with_negative_price_change(df):
    d = df.T.to_dict().values()
    negative_price_changes = 0
    negative_price_changes_ids = {}
    positive_price_changes = 0
    positive_price_changes_ids = {}
    no_price_change = 0

    for row in d:
        if row["abs_price"] < row["prev_price"]:
            negative_price_changes += 1 
            negative_price_changes_ids[row["url"]]=1

        elif row["abs_price"] > row["prev_price"]:
            if row["url"]:
                positive_price_changes += 1 
                positive_price_changes_ids[row["url"]]=1
        else:
            no_price_change += 1 

    for k,v in positive_price_changes_ids.items():
        negative_price_changes_ids.pop(k, None)


    negative_price_changes_ids
    negative_ids=[]

    for k,v in negative_price_changes_ids.items():
        negative_ids.append(k)

    negative_ids_dict = {"url":negative_ids}

    return negative_ids_dict

negative_ids_dict = get_items_with_negative_price_change(df)

filtered_df = df[df['url'].isin(negative_ids_dict['url'])]


import awswrangler as wr
import pandas as pd


# Storing data on Data Lake
wr.s3.to_parquet(
    df=filtered_df,
    path="s3://kv-analysis/price_changes_1/",
    dataset=True,
    database="sandbox",
    table="price_changes"
)




{'paths': ['s3://kv-analysis/price_changes_1/d41151ef5f3f49f6a1853bc73b105394.snappy.parquet'],
 'partitions_values': {}}